In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt
import seaborn as sns
import os
from scipy.stats import kurtosis, skew
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scikit_posthocs as sp
from scipy.stats import wilcoxon

In [2]:
tempTopDir = '/Volumes/Britt_SSD/ReDoEvals3/UserStudy/'
filePath = os.path.join(tempTopDir, 'GP_Main.parquet')

filePath = os.path.join(tempTopDir, 'likertSlim_corrected.parquet')
likertSlim = pd.read_parquet(filePath)


likertSlim['visuals'] = np.where(likertSlim['visuals']=="Visuals","Charts",
                        np.where(likertSlim['visuals']=="Blur_Plus_Vis","Blur+Charts",likertSlim['visuals']))

likertSlim['ag_correctedPerc'] = likertSlim['1_2_correctedPerc']
print(len(likertSlim))

232


In [3]:
# drop particpants who have not been accepted (rejected or timed out)
pID_removed1 = likertSlim[(likertSlim['status']!='APPROVED')].ParticipantPublicID.unique()
print(len(likertSlim))
# drop these given row
# indexes from dataFrame
likertSlim = likertSlim[~likertSlim['ParticipantPublicID'].isin(pID_removed1)].copy()
print(len(likertSlim))
# tmp_slimDF = slimDF[~slimDF['ParticipantPublicID'].isin(pID_removed1)].copy()
# print(tmp_slimDF.status.unique())
# print(pID_removed1)
    
# tmp_demoDF_L = demoDF_L[~demoDF_L['ParticipantPublicID'].isin(pID_removed1)].copy()
# tmp_demoDF_S = demoDF_S[~demoDF_S['ParticipantPublicID'].isin(pID_removed1)].copy()

# tmp_consentDF_L = consentDF_L[~consentDF_L['ParticipantPublicID'].isin(pID_removed1)].copy()
# tmp_consentDF_S = consentDF_S[~consentDF_S['ParticipantPublicID'].isin(pID_removed1)].copy()

# tmp_aiDF_L = aiDF_L[~aiDF_L['ParticipantPublicID'].isin(pID_removed1)].copy()
# tmp_aiDF_S = aiDF_S[~aiDF_S['ParticipantPublicID'].isin(pID_removed1)].copy()

# tmp_taskDF = taskDF[~taskDF['ParticipantPublicID'].isin(pID_removed1)].copy()

# print(tmp_taskDF[tmp_taskDF['ParticipantPublicID']==pID_removed1[-3]])

232
232


In [4]:
df = likertSlim.copy()

for vis in df.visuals.unique():
    print(str(vis) + " " + str(len(df[df['visuals']==vis])))

Blur+Charts 54
Charts 58
Blurred 64
Video 56


In [5]:
vis = sorted(df['visuals'].unique().tolist())


In [6]:
print(vis)

['Blur+Charts', 'Blurred', 'Charts', 'Video']


In [7]:
# function to make statsmodel into dataframe
def results_summary_to_dataframe(results):
    '''take the result of an statsmodel results table and transforms it into a dataframe'''
    print(dir(results))
    pvals = results.pvalues
    coeff = results.params
    conf_lower = results.conf_int()[0]
    conf_higher = results.conf_int()[1]

    results_df = pd.DataFrame({"pvals":pvals,
                               "coeff":coeff,
                               "conf_lower":conf_lower,
                               "conf_higher":conf_higher
                                })

    #Reordering...
    results_df = results_df[["coeff","pvals","conf_lower","conf_higher"]]
    return results_df

In [8]:
def negative_bold(val):
    bold = 'bold' if val < 0 else ''
    return 'font-weight: %s' % bold

In [9]:
likertSlim['OneTwoCorrPerc'] = likertSlim['1_2_WO_Ghosts_correctedPerc']

In [10]:
######################################################################################
######################################################################################
######################################################################################
######################################################################################
######################################################################################
######################################################################################
######################################################################################
######################################################################################
# Part One
######################################################################################
######################################################################################
######################################################################################
######################################################################################
######################################################################################
######################################################################################
######################################################################################

# Part One: 

df = likertSlim.copy()
# df = likertSlim[likertSlim['difficulty']=="Easy"].copy()
# df = likertSlim[likertSlim['difficulty']=="Difficult"].copy()

# ANOVA on totPerc v visuals
# vis = sorted(df['visuals'].unique())
print("predictions")
mod = ols('pred_correctedNum ~ C(visuals)', data=df).fit()
aov_table = sm.stats.anova_lm(mod, typ=2)
print("ANOVA")
print(aov_table)
# print(mod.summary())
print(mod.mse_resid)

# Pair-wise t-test (independent) on totPerc v visuals
pair_t = mod.t_test_pairwise('C(visuals)', method="holm")
print('pair_t')
# print(pair_t)
print("{{{{{{{}}}}}}}")
pairwiseResults = pair_t.result_frame
pairwiseResults['new'] = pairwiseResults.index
new = pairwiseResults.new.str.split("-", n = 3, expand = True)
pairwiseResults['A'] = new[0]
pairwiseResults['B'] = new[1]
pairwiseResults.set_index('A', inplace=True)
pairwiseResults.drop(columns=['Conf. Int. Low','Conf. Int. Upp.','P>|t|','new', 'reject-holm'], inplace=True)
pairwiseResults = pairwiseResults[['B', 'coef' ,  'std err' ,  't'  , 'pvalue-holm']]
pairwiseResults = pairwiseResults.round(decimals=3)
# print(pairwiseResults)
# for i in pairwiseResults.columns.tolist():
#     print(pairwiseResults[i].values)
print(pairwiseResults)

print("\nregions")
mod = ols('reg_correctedNum ~ C(visuals)', data=df).fit()
aov_table = sm.stats.anova_lm(mod, typ=2)
print("ANOVA")
print(aov_table)
# print(mod.summary())
print(mod.mse_resid)

# Pair-wise t-test (independent) on totPerc v visuals
pair_t = mod.t_test_pairwise('C(visuals)', method="holm")
print('pair_t')
# print(pair_t)
print("{{{{{{{}}}}}}}")
pairwiseResults = pair_t.result_frame
pairwiseResults['new'] = pairwiseResults.index
new = pairwiseResults.new.str.split("-", n = 3, expand = True)
pairwiseResults['A'] = new[0]
pairwiseResults['B'] = new[1]
pairwiseResults.set_index('A', inplace=True)
pairwiseResults.drop(columns=['Conf. Int. Low','Conf. Int. Upp.','P>|t|','new', 'reject-holm'], inplace=True)
pairwiseResults = pairwiseResults[['B', 'coef' ,  'std err' ,  't'  , 'pvalue-holm']]
pairwiseResults = pairwiseResults.round(decimals=3)
# print(pairwiseResults)
# for i in pairwiseResults.columns.tolist():
#     print(pairwiseResults[i].values)
print(pairwiseResults)

print("\nlevels")
mod = ols('level_correctedNum ~ C(visuals)', data=df).fit()
aov_table = sm.stats.anova_lm(mod, typ=2)
print("ANOVA")
print(aov_table)
# print(mod.summary())
print(mod.mse_resid)

# Pair-wise t-test (independent) on totPerc v visuals
pair_t = mod.t_test_pairwise('C(visuals)', method="holm")
print('pair_t')
# print(pair_t)
print("{{{{{{{}}}}}}}")
pairwiseResults = pair_t.result_frame
pairwiseResults['new'] = pairwiseResults.index
new = pairwiseResults.new.str.split("-", n = 3, expand = True)
pairwiseResults['A'] = new[0]
pairwiseResults['B'] = new[1]
pairwiseResults.set_index('A', inplace=True)
pairwiseResults.drop(columns=['Conf. Int. Low','Conf. Int. Upp.','P>|t|','new', 'reject-holm'], inplace=True)
pairwiseResults = pairwiseResults[['B', 'coef' ,  'std err' ,  't'  , 'pvalue-holm']]
pairwiseResults = pairwiseResults.round(decimals=3)
# print(pairwiseResults)
# for i in pairwiseResults.columns.tolist():
#     print(pairwiseResults[i].values)
print(pairwiseResults)

print("\nghosts")
mod = ols('ghosts_correctedNum ~ C(visuals)', data=df).fit()
aov_table = sm.stats.anova_lm(mod, typ=2)
print("ANOVA")
print(aov_table)
# print(mod.summary())
print(mod.mse_resid)

# Pair-wise t-test (independent) on totPerc v visuals
pair_t = mod.t_test_pairwise('C(visuals)', method="holm")
print('pair_t')
# print(pair_t)
print("{{{{{{{}}}}}}}")
pairwiseResults = pair_t.result_frame
pairwiseResults['new'] = pairwiseResults.index
new = pairwiseResults.new.str.split("-", n = 3, expand = True)
pairwiseResults['A'] = new[0]
pairwiseResults['B'] = new[1]
pairwiseResults.set_index('A', inplace=True)
pairwiseResults.drop(columns=['Conf. Int. Low','Conf. Int. Upp.','P>|t|','new', 'reject-holm'], inplace=True)
pairwiseResults = pairwiseResults[['B', 'coef' ,  'std err' ,  't'  , 'pvalue-holm']]
pairwiseResults = pairwiseResults.round(decimals=3)
# print(pairwiseResults)
# for i in pairwiseResults.columns.tolist():
#     print(pairwiseResults[i].values)
print(pairwiseResults)

predictions
ANOVA
                sum_sq     df         F    PR(>F)
C(visuals)   42.097830    3.0  8.699512  0.000017
Residual    367.771782  228.0       NaN       NaN
1.6130341335846299
pair_t
{{{{{{{}}}}}}}
                   B   coef  std err      t  pvalue-holm
A                                                       
Blurred  Blur+Charts -0.393    0.235 -1.676        0.285
Charts   Blur+Charts  0.748    0.240  3.114        0.008
Video    Blur+Charts -0.093    0.242 -0.382        0.703
Charts       Blurred  1.141    0.230  4.957        0.000
Video        Blurred  0.301    0.232  1.294        0.394
Video         Charts -0.841    0.238 -3.532        0.002

regions
ANOVA
                sum_sq     df          F        PR(>F)
C(visuals)   36.178114    3.0  15.571133  3.011783e-09
Residual    176.579099  228.0        NaN           NaN
0.774469731813831
pair_t
{{{{{{{}}}}}}}
                   B   coef  std err      t  pvalue-holm
A                                                       
B

In [13]:
#tot_correctedPerc
######################################################################################
######################################################################################
######################################################################################
######################################################################################
######################################################################################
######################################################################################
######################################################################################
######################################################################################
# Part One
######################################################################################
######################################################################################
######################################################################################
######################################################################################
######################################################################################
######################################################################################
######################################################################################

# Part One: 

df = likertSlim.copy()
# df = likertSlim[likertSlim['difficulty']=="Easy"].copy()
# df = likertSlim[likertSlim['difficulty']=="Difficult"].copy()

# levene:
print(stats.levene(df[df['visuals']==vis[0]]['tot_correctedPerc'],\
                      df[df['visuals']==vis[1]]['tot_correctedPerc'],\
                         df[df['visuals']==vis[2]]['tot_correctedPerc'],\
                            df[df['visuals']==vis[3]]['tot_correctedPerc']))

# shapiro:
print(stats.shapiro(df['tot_correctedPerc']))

# ANOVA on totPerc v visuals
# vis = sorted(df['visuals'].unique())

mod = ols('tot_correctedPerc ~ C(visuals)', data=df).fit()
aov_table = sm.stats.anova_lm(mod, typ=2)
print("ANOVA")
print(aov_table)
# print(mod.summary())
print(mod.mse_resid)

# Pair-wise t-test (independent) on totPerc v visuals
pair_t = mod.t_test_pairwise('C(visuals)', method="holm")
print('pair_t')
# print(pair_t)
print("{{{{{{{}}}}}}}")
pairwiseResults = pair_t.result_frame
pairwiseResults['new'] = pairwiseResults.index
new = pairwiseResults.new.str.split("-", n = 3, expand = True)
pairwiseResults['A'] = new[0]
pairwiseResults['B'] = new[1]
pairwiseResults.set_index('A', inplace=True)
pairwiseResults.drop(columns=['Conf. Int. Low','Conf. Int. Upp.','P>|t|','new', 'reject-holm'], inplace=True)
pairwiseResults = pairwiseResults[['B', 'coef' ,  'std err' ,  't'  , 'pvalue-holm']]
pairwiseResults = pairwiseResults.round(decimals=3)
print(pairwiseResults)
for i in pairwiseResults.columns.tolist():
    print(pairwiseResults[i].values)

# for vis in df.visuals.unique():
#     tmp = df[df['visuals']==vis]
#     print("\n" + str(vis) + " " + str(len(tmp)))
#     print("Mean: " + str(stats.tmean(tmp['totPerc'])))
#     print("StdDev: " + str(stats.tstd(tmp['totPerc'])))

LeveneResult(statistic=2.050701234242531, pvalue=0.1076024107312195)
ShapiroResult(statistic=0.9760276079177856, pvalue=0.0005678394227288663)
ANOVA
                  sum_sq     df          F        PR(>F)
C(visuals)  11456.314156    3.0  22.769785  6.244127e-13
Residual    38238.387721  228.0        NaN           NaN
167.7122268444748
pair_t
{{{{{{{}}}}}}}
                   B    coef  std err      t  pvalue-holm
A                                                        
Blurred  Blur+Charts  -7.552    2.393 -3.156        0.005
Charts   Blur+Charts  11.284    2.449  4.608        0.000
Video    Blur+Charts  -2.749    2.470 -1.113        0.267
Charts       Blurred  18.835    2.348  8.023        0.000
Video        Blurred   4.803    2.370  2.027        0.088
Video         Charts -14.033    2.426 -5.784        0.000
['Blur+Charts' 'Blur+Charts' 'Blur+Charts' 'Blurred' 'Blurred' 'Charts']
[ -7.552  11.284  -2.749  18.835   4.803 -14.033]
[2.393 2.449 2.47  2.348 2.37  2.426]
[-3.156  4.608 

In [14]:
# # Part One: 

# # Ignore, we talk about this later

# # df = likertSlim.copy()
# df = likertSlim[likertSlim['difficulty']=="Easy"].copy()
# # df = likertSlim[likertSlim['difficulty']=="Difficult"].copy()

# # ANOVA on totPerc v visuals
vis = sorted(df['visuals'].unique())

mod = ols('tot_correctedPerc ~ C(visuals)', data=df).fit()
aov_table = sm.stats.anova_lm(mod, typ=2)
print(aov_table)
# print(mod.summary())
print(mod.mse_resid)


# Pair-wise t-test (independent) on totPerc v visuals
pair_t = mod.t_test_pairwise('C(visuals)', method="holm")
print(pair_t)
print("{{{{{{{}}}}}}}")
print(pair_t.result_frame)

# # Part One: 
# # Ignore, we talk about this later

# # df = likertSlim.copy()
# # df = likertSlim[likertSlim['difficulty']=="Easy"].copy()
# df = likertSlim[likertSlim['difficulty']=="Difficult"].copy()

# # ANOVA on totPerc v visuals
# vis = sorted(df['visuals'].unique())

# mod = ols('totPerc ~ C(visuals)', data=df).fit()
# aov_table = sm.stats.anova_lm(mod, typ=2)
# print(aov_table)
# # print(mod.summary())
# print(mod.mse_resid)

# # Pair-wise t-test (independent) on totPerc v visuals
# pair_t = mod.t_test_pairwise('C(visuals)', method="holm")
# print(pair_t)
# print("{{{{{{{}}}}}}}")
# print(pair_t.result_frame)

                  sum_sq     df          F        PR(>F)
C(visuals)  11456.314156    3.0  22.769785  6.244127e-13
Residual    38238.387721  228.0        NaN           NaN
167.7122268444748
{{{{{{{}}}}}}}
                          coef   std err         t         P>|t|  \
Blurred-Blur+Charts  -7.551732  2.392966 -3.155804  1.816350e-03   
Charts-Blur+Charts   11.283571  2.448953  4.607507  6.781056e-06   
Video-Blur+Charts    -2.749197  2.469948 -1.113059  2.668554e-01   
Charts-Blurred       18.835302  2.347785  8.022585  5.423773e-14   
Video-Blurred         4.802535  2.369676  2.026663  4.386110e-02   
Video-Charts        -14.032767  2.426201 -5.783844  2.391472e-08   

                     Conf. Int. Low  Conf. Int. Upp.   pvalue-holm  \
Blurred-Blur+Charts      -12.266887        -2.836577  5.449051e-03   
Charts-Blur+Charts         6.458096        16.109045  2.712422e-05   
Video-Blur+Charts         -7.616040         2.117646  2.668554e-01   
Charts-Blurred            14.209173    

In [15]:
# Part One: 
vis = sorted(df['visuals'].unique())
print(vis)
df = likertSlim.copy()
# df = likertSlim[likertSlim['difficulty']=="Easy"].copy()
# df = likertSlim[likertSlim['difficulty']=="Difficult"].copy()

# Kruskal-Wallis on totPerc v visuals
print("And kw: ")
stats.kruskal(df[df['visuals']==vis[0]]['tot__WO_Ghosts_correctedPerc'],
               df[df['visuals']==vis[1]]['tot__WO_Ghosts_correctedPerc'],
               df[df['visuals']==vis[2]]['tot__WO_Ghosts_correctedPerc'],
               df[df['visuals']==vis[3]]['tot__WO_Ghosts_correctedPerc'])


['Blur+Charts', 'Blurred', 'Charts', 'Video']
And kw: 


KeyError: 'tot__WO_Ghosts_correctedPerc'

In [16]:
# Part One: 
vis = sorted(df['visuals'].unique())
print(vis)
df = likertSlim.copy()
# df = likertSlim[likertSlim['difficulty']=="Easy"].copy()
# df = likertSlim[likertSlim['difficulty']=="Difficult"].copy()

# Kruskal-Wallis on totPerc v visuals
print("And kw: ")
stats.kruskal(df[df['visuals']==vis[0]]['tot_correctedPerc'],
               df[df['visuals']==vis[1]]['tot_correctedPerc'],
               df[df['visuals']==vis[2]]['tot_correctedPerc'],
               df[df['visuals']==vis[3]]['tot_correctedPerc'])


['Blur+Charts', 'Blurred', 'Charts', 'Video']
And kw: 


KruskalResult(statistic=52.777066540925404, pvalue=2.045385095366996e-11)

In [17]:
# Part One: 

# df = likertSlim.copy()
df = likertSlim[likertSlim['difficulty']=="Easy"].copy()
# df = likertSlim[likertSlim['difficulty']=="Difficult"].copy()

# Kruskal-Wallis on totPerc v visuals
print("And kw: ")
stats.kruskal(df[df['visuals']==vis[0]]['totPerc'],
               df[df['visuals']==vis[1]]['totPerc'],
               df[df['visuals']==vis[2]]['totPerc'],
               df[df['visuals']==vis[3]]['totPerc'])




And kw: 


KruskalResult(statistic=17.82903458991321, pvalue=0.000477041685861493)

In [18]:
# Part One: 

# df = likertSlim.copy()
df = likertSlim[likertSlim['difficulty']=="Easy"].copy()
# df = likertSlim[likertSlim['difficulty']=="Difficult"].copy()

# Kruskal-Wallis on totPerc v visuals
print("And kw: ")
stats.kruskal(df[df['visuals']==vis[0]]['tot_correctedPerc'],
               df[df['visuals']==vis[1]]['tot_correctedPerc'],
               df[df['visuals']==vis[2]]['tot_correctedPerc'],
               df[df['visuals']==vis[3]]['tot_correctedPerc'])





And kw: 


KruskalResult(statistic=9.985979994384888, pvalue=0.018685687361040378)

In [19]:
# Part One: 

# df = likertSlim.copy()
# df = likertSlim[likertSlim['difficulty']=="Easy"].copy()
df = likertSlim[likertSlim['difficulty']=="Difficult"].copy()

# Kruskal-Wallis on totPerc v visuals
print("And kw: ")
stats.kruskal(df[df['visuals']==vis[0]]['totPerc'],
               df[df['visuals']==vis[1]]['totPerc'],
               df[df['visuals']==vis[2]]['totPerc'],
               df[df['visuals']==vis[3]]['totPerc'])




And kw: 


KruskalResult(statistic=42.570699621983735, pvalue=3.0356371753057677e-09)

In [20]:
# Part One: 

# df = likertSlim.copy()
# df = likertSlim[likertSlim['difficulty']=="Easy"].copy()
df = likertSlim[likertSlim['difficulty']=="Difficult"].copy()

# Kruskal-Wallis on totPerc v visuals
print("And kw: ")
stats.kruskal(df[df['visuals']==vis[0]]['tot_correctedPerc'],
               df[df['visuals']==vis[1]]['tot_correctedPerc'],
               df[df['visuals']==vis[2]]['tot_correctedPerc'],
               df[df['visuals']==vis[3]]['tot_correctedPerc'])





And kw: 


KruskalResult(statistic=43.21753323838556, pvalue=2.2127042123237364e-09)

In [21]:
# Mean and STD of totPerc after correction for guessing... 

for v in vis:
    tmp = df[df['visuals']==v]
    print(v)
    print(tmp['tot_correctedPerc'].mean())
    print(tmp['tot_correctedPerc'].std())
    print()

Blur+Charts
24.821428571428577
13.376665697593896

Blurred
16.484126984126984
10.027427562986242

Charts
42.07488479262673
14.292963502491839

Video
21.232804232804234
12.940504974396605



In [22]:
print("\n Qs1_Corrected")
print(df['Qs1_CorrectedPerc'].mean())
print(df['Qs1_CorrectedPerc'].std())

print("\n Qs2_Corrected")
print(df['Qs2_CorrectedPerc'].mean())
print(df['Qs2_CorrectedPerc'].std())

print("\n 1_2_correctedPerc")
print(df['1_2_correctedPerc'].mean())
print(df['1_2_correctedPerc'].std())

print("\n comp_correctedPerc")
print(df['comp_correctedPerc'].mean())
print(df['comp_correctedPerc'].std())

print("\n tot_correctedPerc")
print(df['tot_correctedPerc'].mean())
print(df['tot_correctedPerc'].std())





 Qs1_Corrected
24.78410421545668
18.2210407185097

 Qs2_Corrected
24.929742388758783
18.55438959543301

 1_2_correctedPerc
24.85692330210773
15.62050865045431

 comp_correctedPerc
30.327868852459016
36.11127033223079

 tot_correctedPerc
25.95111241217798
15.942963014286152
